# 作业1

以上证指数的收盘价作为分析指标

<br>

## 收集数据
这里定义getData方法获取上证指数，同时将数据集划分为训练集和测试集分开记录

In [ ]:
import akshare as ak
import pandas as pd

def getData():
    dailyTrain_df = ak.stock_zh_index_daily_em(symbol="sh000001", start_date="20000101", end_date="20181231")
    dailyTest_df = ak.stock_zh_index_daily_em(symbol="sh000001", start_date="20190101", end_date="20191231")
    """
    训练数据
    """
    # 保存为文本&csv
    with open("train.txt", "w", encoding="utf-8") as f:
        f.write(dailyTrain_df.to_string(index=False))
    dailyTrain_df.to_csv("train.csv", index=False, encoding="utf-8")
    """
    测试数据
    """
    with open("test.txt", "w", encoding="utf-8") as f:
        f.write(dailyTest_df.to_string(index=False))
    dailyTest_df.to_csv("test.csv", index=False, encoding="utf-8")
    
if __name__ == "__main__":
    getData()

## 数据离散化

清理出涨跌幅在-5%到5%的数据并重新保存数据

In [ ]:
# 实现离散化
def cleanData(path):
    df = pd.read_csv(path)
    # 对应前一次的收盘数据
    df['prev_close'] = df['close'].shift(1)
    # 百分比计算
    df['pct_change'] = (df['close'] - df['prev_close'])/df['close']
    # 筛选
    filtered_df = df[(-0.05 <= df['pct_change']) & (df['pct_change'] <= 0.05)]
    result_df = filtered_df[['date', 'close', 'pct_change']]
    result_df.to_csv("cleaned " + path, index=False)

if __name__ == '__main__':
    cleanData("train.csv")
    cleanData("test.csv")

## 频率统计

按照涨跌幅数据分割为5个状态 ： 大跌， 下跌， 震荡， 上涨， 大涨

对应的幅度为 [-5%, -3%), [-3%, -1%), [-1%, 1%), [1%, 3%), [3%, 5%]

统计出对应的频率，并得到转移矩阵，这里使用了热力图去展示结果

In [ ]:
import matplotlib as plt
import seaborn as sns
def status():
        df = pd.read_csv('Dr_Guan/cleaned_train.csv')
        bins = [-0.05, -0.03, -0.01, 0.01, 0.03, 0.05]
        labels = ["大跌","下跌", "震荡", "上涨", "大涨"]
        # 打标签
        df['state'] = pd.cut(df['pct_change'], bins=bins, labels=labels)
        # 构造状态转移序列（前一状态 -> 当前状态）
        df['prev_state'] = df['state'].shift(1)
        # 去掉shift影响的第一行
        df_cleaner = df.dropna(subset=['state', 'prev_state'])
        # 构建状态转移计数矩阵
        transition_counts = pd.crosstab(df_cleaner['prev_state'], df_cleaner['state'])
        # 构建转移概率矩阵（按行归一化）
        global  transition_matrix
        transition_matrix = transition_counts.div(transition_counts.sum(axis=1), axis=0)
        # 展示转移概率矩阵
        plt.figure(figsize=(8, 6))
        sns.heatmap(transition_matrix, annot=True, fmt=".2f", cmap="Blues", cbar=True, linewidths=0.5)
        plt.title("马尔可夫链状态转移概率矩阵", fontsize=14)
        plt.xlabel("当前状态", fontsize=12)
        plt.ylabel("前一状态", fontsize=12)

        plt.tight_layout()
        plt.show()
if __name__ == '__main__':
    status()

最终可以得到转移矩阵如上图所示

## 转移矩阵实验

输出对应的转移矩阵数据，观察其是否收敛，多久收敛，这里使用迭代法去完成

In [ ]:
    import numpy as np
    def check():
        """
        实现收敛分析，这里采用迭代法
        """
        T = transition_matrix.values
        # 设置初始状态
        v = np.zeros(T.shape[0])
        v[0] = 1.0
        tolerance = 1e-6
        max_steps = 500
        history = [v.copy()]

        for i in range(max_steps):
            v_next = v @ T
            history.append(v_next)
            if np.allclose(v_next, v, atol=tolerance):
                print(f"收敛于第 {i + 1} 步")
                break
            v = v_next
        else:
            print("未收敛")
        # 打印稳态分布
        print("稳态分布向量：", v_next.round(4))
        history = np.array(history)
        plt.figure(figsize=(10, 6))
        for i in range(history.shape[1]):
            plt.plot(history[:, i], label=f"状态 {transition_matrix.columns[i]}")

        plt.xlabel("步数")
        plt.ylabel("状态概率")
        plt.title("马尔可夫链状态概率收敛过程")
        plt.legend()
        plt.grid()
        plt.tight_layout()
        plt.show()

最终得到的结果如下：

收敛于第 7 步

稳态分布向量： [0.0227 0.1485 0.6314 0.1736 0.0238]

## 改变参数看对比

1.  改变时间结点
2.  改变状态定义